<hr style="color:#5A7D9F;">
<p align="center">
    <b style="font-size:2vw; color:#5A7D9F; font-weight:bold;">
    <center>Understanding Ocean Deoxygenation</center>
    </b>
</p>
<hr style="color:#5A7D9F;">

Do not forget to activate the **environnement esa_diffusion**

In [ ]:
# ---------- Librairies ----------
import matplotlib.pyplot as plt

# Pytorch

# ---------- Jupyter ----------
%matplotlib inline
plt.rcParams.update({"font.size": 13})

# Making sure modules are reloaded when modified
%reload_ext autoreload
%autoreload 2

# Moving to code folder
%cd poseidon/

- Can you help me simplify the code where tis possible ? Avoid repetitions for example, …
- Change variable / functions name for clarity if you think its necessary ?
- Complete docstrings where needed ?
- Only add comments # if you really think they are necessary to understand, otherwise remove it

<hr style="color:#5A7D9F;">
<p align="center">
    <b style="font-size:2vw; color:#5A7D9F; font-weight:bold;">
    <center>Playground</center>
    </b>
</p>
<hr style="color:#5A7D9F;">

In [5]:
# 1. Get Access to the data
#
# 2. Filter out:
#    - Variables
#    - Time
#
# 3. Compute:
#    - Mean
#    - Standard deviation
#
# 4. Save as a zarr
#
# Create a script that uses the tools to compute the statistics

In [2]:
import xarray as xr

from poseidon.data.const import (
    DATASET_TRAINING_DATE_END,
    DATASET_TRAINING_DATE_START,
    DATASET_VARIABLES,
)
from poseidon.data.statistics import PoseidonStatistics, preprocess_dataset
from poseidon.utils import generate_paths

# isort: split
from poseidon.config import POSEIDON_MASK, POSEIDON_STAT

# ----------------------------------------------------------------
#                           SCRIPT
# ----------------------------------------------------------------
#
#
# Generate the paths and Iiitialize the statistics tool
paths_dictionary = generate_paths()
stats_tool = PoseidonStatistics()

# Used for working on correct time range
time_skipping = True

for date, path in paths_dictionary.items():
    if date == DATASET_TRAINING_DATE_START:
        time_skipping = False
    if time_skipping:
        continue
    if date == DATASET_TRAINING_DATE_END:
        time_skipping = True

    # Load and preprocess (mask, variables, ...) sample
    dataset = preprocess_dataset(
        dataset=xr.open_mfdataset(path, combine="by_coords", engine="netcdf4", parallel=True),
        mask=xr.open_zarr(POSEIDON_MASK),
        variables=DATASET_VARIABLES,
    )

    # Update the statistics
    stats_tool.update(dataset)

    print(date)
    break

# Save the statistics
stats_dataset = xr.concat(
    [stats_tool.get_mean().load(), stats_tool.get_standard_deviation().load()], dim="statistic"
)
stats_dataset = stats_dataset.assign_coords(statistic=["mean", "std"])
stats_dataset.attrs["Date (Start)"] = DATASET_TRAINING_DATE_START
stats_dataset.attrs["Date (End)"] = DATASET_TRAINING_DATE_END
stats_dataset.to_zarr(POSEIDON_STAT, mode="w")

1995-01


/gpfs/home/acad/ulg-mast/vmangele/micromamba/envs/esa_diffusion/lib/python3.9/site-packages/dask/core.py:127: RuntimeWarning: invalid value encountered in divide
  return func(*(_execute_task(a, cache) for a in args))
/gpfs/home/acad/ulg-mast/vmangele/micromamba/envs/esa_diffusion/lib/python3.9/site-packages/dask/core.py:127: RuntimeWarning: invalid value encountered in divide
  return func(*(_execute_task(a, cache) for a in args))
/gpfs/home/acad/ulg-mast/vmangele/micromamba/envs/esa_diffusion/lib/python3.9/site-packages/dask/core.py:127: RuntimeWarning: invalid value encountered in sqrt
  return func(*(_execute_task(a, cache) for a in args))


In [4]:
test = xr.open_zarr(POSEIDON_STAT)

In [ ]:
test

In [3]:
stats_dataset

<xarray.Dataset> Size: 6kB
Dimensions:    (statistic: 2, level: 59)
Coordinates:
  * level      (level) float32 236B 0.2546 0.7742 1.316 ... 2.287e+03 2.434e+03
  * statistic  (statistic) <U4 32B 'mean' 'std'
Data variables:
    DOX        (statistic, level) float64 944B 319.7 319.7 319.7 ... 0.01183 nan
    CHL        (statistic, level) float64 944B 2.45 2.451 ... 1.862e-16 nan
    ssh        (statistic, level) float64 944B -0.07408 -0.07408 ... 0.04649 nan
    votemper   (statistic, level) float64 944B 7.652 7.652 ... 0.002186 nan
    vosaline   (statistic, level) float64 944B 18.24 18.24 18.24 ... nan nan nan
    rho        (statistic, level) float64 944B 1.014e+03 1.014e+03 ... 0.112 nan
Attributes:
    Date (Start):  1995-01
    Date (End):    2015-12